<a href="https://colab.research.google.com/github/Antonio14-code/agenteIA/blob/main/Acciones_de_Agente_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform

In [ ]:
# Restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

In [ ]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [ ]:

import vertexai

PROJECT_ID = "my-proyecto-conauti-01"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}


vertexai.init(project=PROJECT_ID, location=LOCATION)

In [ ]:
import json
import sys
import traceback
from typing import Callable, Tuple

from google.protobuf.json_format import MessageToJson

from vertexai import generative_models
from vertexai.generative_models import (
    FunctionDeclaration, #Function Calling
    GenerativeModel,
    GenerationConfig,
    Part, #Function Calling
    Tool, #Function Calling
)

#Configurar el modo de funcionamiento de Function Calling
from vertexai.preview.generative_models import (
    ToolConfig #Function Calling
)

In [ ]:
#Configurar Nuestro Modelo

#Seguridad
Configuracion_Seguridad = {
    vertexai.preview.generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: vertexai.preview.generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    vertexai.preview.generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: vertexai.preview.generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    vertexai.preview.generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: vertexai.preview.generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    vertexai.preview.generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: vertexai.preview.generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

#Configuración
configuracion_Modelo = {
    "max_output_tokens": 8192,
    "temperature":0.2,
    #"top_k":10,
    "top_p":0.95
}

In [ ]:
model = GenerativeModel(
    "gemini-1.5-pro-001",
    system_instruction=[
        "You are an assistant that helps me tidy my room."
        "Your goal is to make sure all the books are on the shelf, all clothes are in the hamper, and the trash is empty.",
        "You cannot receive any input from me.",
    ],
    generation_config=configuracion_Modelo,
    safety_settings=Configuracion_Seguridad,
#    safety_settings=[
#        generative_models.SafetySetting(
#            category=generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
#            method=generative_models.SafetySetting.HarmBlockMethod.PROBABILITY,
#            threshold=generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
#        ),
#],
)

In [ ]:
verbose = True

In [ ]:
# Conveience function to print multiline text indented
def indent(text, amount, ch=" "):
    padding = amount * ch
    return "".join(padding + line for line in text.splitlines(True))


# Convenience function for logging statements
def logging(msg):
    global verbose
    print(msg) if verbose else None


# Retrieve the text from a model response
def get_text(resp):
    return resp.candidates[0].content.parts[0].text


# Retrieve the function call information from a model response
def get_function_call(resp):
    return resp.candidates[0].function_calls[0]


def get_action_label(json_payload, log, role="MODEL"):
    log(f"{role}: {json_payload}")
    answer = json.loads(json_payload)
    action = answer["next_action"]
    return action


def get_action_from_function_call(func_payload, log, role="MODEL"):
    json_payload = MessageToJson(func_payload._pb)
    log(f"{role}: {json_payload}")
    return func_payload.name

In [ ]:
# Initial room state
def reset_room_state(room_state):
    room_state.clear()
    room_state["clothes"] = "floor"
    room_state["books"] = "scattered"
    room_state["wastebin"] = "empty"


# Functions for actions (replace these with Gemini function calls)
def pick_up_clothes(room_state):
    room_state["clothes"] = "carrying by hand"
    return room_state, "The clothes are now being carried."


def put_clothes_in_hamper(room_state):
    room_state["clothes"] = "hamper"
    return room_state, "The clothes are now in the hamper."


def pick_up_books(room_state):
    room_state["books"] = "in hand"
    return room_state, "The books are now in my hand."


def place_books_on_shelf(room_state):
    room_state["books"] = "shelf"
    return room_state, "The books are now on the shelf."


def empty_wastebin(room_state):
    room_state["wastebin"] = "empty"
    return room_state, "The wastebin is emptied."


# Maps a function string to its respective function reference.
def get_func(action_label):
    return None if action_label == "" else getattr(sys.modules[__name__], action_label)

In [ ]:
# Function to check if the room is tidy
# Some examples below do not call this function,
# for those examples the model takes on the goal validation role.
def is_room_tidy(room_state):
    return all(
        [
            room_state["clothes"] == "hamper",
            room_state["books"] == "shelf",
            room_state["wastebin"] == "empty",
        ]
    )

In [ ]:
functions = """
<actions>
    put_clothes_in_hamper - place clothes into hamper, instead of carrying them around in your hand.
    pick_up_clothes - pick clothes up from the floor.
    pick_up_books - pick books up from anywhere not on the shelf
    place_books_on_shelf - self explanatory.
    empty_wastebin - self explanatory.
    done - when everything are in the right place.
</actions>"""


def get_next_step_full_prompt(state, cycle, log):
    observation = f"The room is currently in this state: {state}."
    prompt = "\n".join(
        [
            observation,
            f"You can pick any of the following action labels: {functions}",
            "Which one should be the next step to achieve the goal? ",
            'Return a single JSON object containing fields "next_action" and "rationale".',
        ]
    )
    log("PROMPT:\n{}".format(indent(prompt, 1, "\t"))) if cycle == 1 else log(
        f"OBSERVATION: {observation}"
    )

    return prompt

In [ ]:
# Main ReAct loop
def main_react_loop(loop_continues, log):
    room_state = {}
    reset_room_state(room_state)
    trash_added = False

    cycle = 1
    while loop_continues(cycle, room_state):
        log(f"Cycle #{cycle}")

        # Observe the environment (use Gemini to generate an action thought)
        try:  # REASON #
            response = model.generate_content(
                get_next_step_full_prompt(room_state, cycle, log),
                generation_config={"response_mime_type": "application/json"},
            )  # JSON Mode
            action_label = get_action_label(get_text(response).strip(), log)

        except Exception:
            traceback.print_exc()
            log(response)
            break

        # Execute the action and get the observation
        if action_label == "done":
            break

        try:  # ACTION #
            # Call the function mapped from the label
            room_state, acknowledgement = get_func(action_label)(room_state)
            log(f"ACTION:   {action_label}\nEXECUTED: {acknowledgement}\n")

        except Exception:
            log("No action suggested.")

        # Simulating a change in environment
        if cycle == 4 and not trash_added:
            room_state["wastebin"] = "1 item"
            trash_added = True

        cycle += 1
        # End of while loop

    # Determine the final result
    result = (
        "The room is tidy!" if is_room_tidy(room_state) else "The room is not tidy!"
    )

    return room_state, result

In [ ]:
#como inicializa el valor del input 1 para la función main_react loop
room_state={}
#print(room_state)
reset_room_state(room_state)
print(f'El cuarto empieza en el estado -> {room_state}')
is_room_tidy(room_state)
c=1
print(f'El ciclo de trabajo del Agente IA empieza en: {c}')
r=room_state
resultado=lambda c, r: c <= 10 and not is_room_tidy(r)
print(f'Vamos a inicializar la funcion loop main_react_loop con : {resultado(c,r)}')

El cuarto empieza en el estado -> {'clothes': 'floor', 'books': 'scattered', 'wastebin': 'empty'}
El ciclo de trabajo del Agente IA empieza en: 1
Vamos a inicializar la funcion loop main_react_loop con : True


In [ ]:
#como inicializa el valor del input 2 para la función main_react loop
print(logging)

<function logging at 0x7de9843979c0>


In [ ]:
# We are passing in a while loop continuation test function:
# Continue while loop when number of cycles <= 10 AND the room is not yet tidy.
# We are explicitly testing if the room is tidy within code.
#
# To save space, only the first cycle prints the full prompt.
# The same prompt template is used for every model call with a modified room state.
room_state, result = main_react_loop(
    lambda c, r: c <= 10 and not is_room_tidy(r), logging
)
print(room_state, result)

Cycle #1
PROMPT:
	The room is currently in this state: {'clothes': 'floor', 'books': 'scattered', 'wastebin': 'empty'}.
	You can pick any of the following action labels: 
	<actions>
	    put_clothes_in_hamper - place clothes into hamper, instead of carrying them around in your hand.
	    pick_up_clothes - pick clothes up from the floor.
	    pick_up_books - pick books up from anywhere not on the shelf
	    place_books_on_shelf - self explanatory.
	    empty_wastebin - self explanatory.
	    done - when everything are in the right place.
	</actions>
	Which one should be the next step to achieve the goal? 
	Return a single JSON object containing fields "next_action" and "rationale".
MODEL: {"next_action": "pick_up_clothes", "rationale": "The clothes are on the floor and need to be picked up before they can be put in the hamper."}
ACTION:   pick_up_clothes
EXECUTED: The clothes are now being carried.

Cycle #2
OBSERVATION: The room is currently in this state: {'clothes': 'carrying by hand

In [ ]:
# We are passing in a while loop continuation test function:
# Continue while loop when number of cycles <= 10
# We are no longer testing if the room is tidy within code.
# The decision is now up to the model.
room_state, result = main_react_loop(lambda c, r: c <= 10, logging)
print(room_state, result)

Cycle #1
PROMPT:
	The room is currently in this state: {'clothes': 'floor', 'books': 'scattered', 'wastebin': 'empty'}.
	You can pick any of the following action labels: 
	<actions>
	    put_clothes_in_hamper - place clothes into hamper, instead of carrying them around in your hand.
	    pick_up_clothes - pick clothes up from the floor.
	    pick_up_books - pick books up from anywhere not on the shelf
	    place_books_on_shelf - self explanatory.
	    empty_wastebin - self explanatory.
	    done - when everything are in the right place.
	</actions>
	Which one should be the next step to achieve the goal? 
	Return a single JSON object containing fields "next_action" and "rationale".
MODEL: {"next_action": "pick_up_clothes", "rationale": "The clothes are on the floor, so we need to pick them up first before we can put them in the hamper."}
ACTION:   pick_up_clothes
EXECUTED: The clothes are now being carried.

Cycle #2
OBSERVATION: The room is currently in this state: {'clothes': 'carryin

In [ ]:
#usamos Gemini
from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    Part,
)

In [ ]:
#TextGenerationModel.from_pretrained("text-bison@001") Descontinuado
multimodal_model = GenerativeModel("gemini-1.5-pro-001")

In [ ]:
generation_config = GenerationConfig(
    temperature=1, #0 - 2
    top_p=0.5,
    #top_k=32, # 1.5 PRO no existe
    candidate_count=1,
    max_output_tokens=4000, #max 8192 tokens
)

In [ ]:
import json

room_state = '{"ropa": "desordenada", "libros": "en el suelo", "juguetes": "esparcidos"}'
result = '{"estado_general": "desordenado"}'

room_state_json = json.loads(room_state)
result_json = json.loads(result)

print("room_state_dict:", room_state_json)
print("result_dict:", result_json)

room_state_dict: {'ropa': 'desordenada', 'libros': 'en el suelo', 'juguetes': 'esparcidos'}
result_dict: {'estado_general': 'desordenado'}


In [ ]:
functions = """
<acciones>
    poner_ropa_en_cesto - colocar la ropa en el cesto, en lugar de llevarla en la mano.
    recoger_ropa - recoger la ropa del suelo.
    recoger_libros - recoger los libros de cualquier lugar que no sea la estantería.
    colocar_libros_en_estanteria - autoexplicativo.
    vaciar_papelera - autoexplicativo.
    listo - cuando todo esté en su lugar correcto.
</acciones>
"""


prompt = f"""
Estado de la habitación: {room_state_json}
Resultado del análisis: {result_json}

Considerando el estado actual de la habitación, ¿cuál sería el orden más eficiente para organizarla?
Utiliza las siguientes acciones para describir los pasos detallados:

{functions}

Proporciona la respuesta en formato de una lista numerada, usando las acciones definidas.
"""

contents = [
    prompt,
]


responses = multimodal_model.generate_content(contents, generation_config=generation_config, stream=True)

print("\n-------Respuesta--------")
for response in responses:
    print(response.text, end="")


-------Respuesta--------
Aquí tienes un posible orden eficiente para organizar la habitación, considerando su estado actual:

1. **`recoger_ropa`**: Es mejor empezar por la ropa, ya que está "desordenada" lo que implica que está esparcida y necesita ser recogida primero.
2. **`poner_ropa_en_cesto`**:  Una vez recogida, se coloca directamente en el cesto para evitar desorden adicional.
3. **`recoger_libros`**:  Los libros en el suelo representan un segundo foco de desorden.
4. **`colocar_libros_en_estanteria`**:  Devolvemos los libros a su lugar.
5. **`vaciar_papelera`**:  Aunque no se menciona explícitamente, vaciar la papelera ayuda a mantener el orden general y suele ser una buena práctica al finalizar la organización.
6. **`listo`**:  ¡Habitación organizada! 
